# Seedhe Maut - Lyric Scraping and DataFrame Creation

This notebook adapts the provided Python script to download all lyrics for selected albums by the Indian hip-hop duo **Seedhe Maut**. It then processes the downloaded text files into a clean `pandas` DataFrame.

**Albums included:**
* Bayaan
* Nayaab
* Lunch Break

### Step 0: Install Necessary Libraries

First, let's make sure all the required libraries are installed. Run the cell below.

In [43]:
!pip install beautifulsoup4 requests lyricsgenius pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Step 1: Import Libraries and Define Core Functions

This section contains all the necessary imports and the functions to get song titles from a Genius album page and download the corresponding lyrics.

In [44]:
import re
import string
import requests
import pandas as pd
import lyricsgenius
from pathlib import Path
from bs4 import BeautifulSoup

def clean_up(song_title):
    """Cleans the raw song title scraped from Genius."""
    if "Ft" in song_title:
        before_ft_pattern = re.compile(r".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")

    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")

    return clean_song_title

def get_all_songs_from_album(artist, album_name):
    """Scrapes Genius for all song titles from a specific album."""
    artist = artist.replace(" ", "-")
    album_name = album_name.replace(" ", "-")

    response = requests.get(f"https://genius.com/albums/{artist}/{album_name}")
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]

    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)

    return clean_songs

def download_album_lyrics(artist, album_name):
    """Downloads lyrics for all songs in an album and saves them as .txt files."""

    # IMPORTANT: You must get your own client access token from Genius Developers
    # Go to https://genius.com/api-clients and create a new API client.
    client_access_token = 'insert_your_client_access_token_here'

    if client_access_token == 'INSERT_YOUR_CLIENT_ACCESS_TOKEN_HERE':
        print("🚨 ERROR: Please insert your Genius client access token before running!")
        return

    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True # Removes headers like [Verse 1], [Chorus], etc.

    print(f"Fetching songs for {album_name}...")
    clean_songs = get_all_songs_from_album(artist, album_name)

    for song in clean_songs:

        song_object = LyricsGenius.search_song(song, artist)

        if song_object:
            artist_title = artist.replace(" ", "-")
            album_title = album_name.replace(" ", "-")
            song_title = song.replace("/", "-").replace(" ", "-")

            # Define folder and file path
            folder_path = Path(f"{artist_title}_{album_title}")
            folder_path.mkdir(parents=True, exist_ok=True)
            custom_filename = folder_path / song_title

            print(f"  -> Downloading lyrics for '{song}'...")
            song_object.save_lyrics(filename=custom_filename, extension='txt', sanitize=False)

        else:
            print(f"  -> Could not find lyrics for '{song}'. Skipping.")

    print(f"Finished downloading lyrics for {album_name}.\n")

### Step 2: Specify Artists + Albums and Download Lyrics

**Action Required:**
1.  **Paste your Genius API key** into the `client_access_token` variable in the function above.
2.  **Uncomment** the lines below to download the lyrics for each Seedhe Maut album.

It's recommended to run them one at a time to avoid potential timeout errors from the Genius API.

In [45]:
# Specify the artist and albums for Seedhe Maut
artist = "Seedhe Maut"
albums = ["Bayaan", "Nayaab", "Lunch Break"]

# Uncomment the lines below to run the download
for album in albums:
    download_album_lyrics(artist, album)

# Example of running for a single album:
# download_album_lyrics("Seedhe Maut", "Bayaan")

Fetching songs for Bayaan...
Searching for "Intro" by Seedhe Maut...
Done.
  -> Downloading lyrics for 'Intro'...
Wrote Seedhe-Maut_Bayaan\Intro.txt.
Searching for "Shaktimaan" by Seedhe Maut...
Done.
  -> Downloading lyrics for 'Shaktimaan'...
Wrote Seedhe-Maut_Bayaan\Shaktimaan.txt.
Searching for "Gehraiyaan" by Seedhe Maut...
Done.
  -> Downloading lyrics for 'Gehraiyaan'...
Wrote Seedhe-Maut_Bayaan\Gehraiyaan.txt.
Searching for "Uss Din" by Seedhe Maut...
Done.
  -> Downloading lyrics for 'Uss Din'...
Wrote Seedhe-Maut_Bayaan\Uss-Din.txt.
Searching for "Jolly (skit)" by Seedhe Maut...
Specified song does not contain lyrics. Rejecting.
  -> Could not find lyrics for 'Jolly (skit)'. Skipping.
Searching for "Meri Baggi" by Seedhe Maut...
Done.
  -> Downloading lyrics for 'Meri Baggi'...
Wrote Seedhe-Maut_Bayaan\Meri-Baggi.txt.
Searching for "Dehshat" by Seedhe Maut...
Done.
  -> Downloading lyrics for 'Dehshat'...
Wrote Seedhe-Maut_Bayaan\Dehshat.txt.
Searching for "Pnp" by Seedhe Mau

### Step 3: Load Text Data into a Clean DataFrame

Once the lyrics are downloaded into folders, this code will read each `.txt` file and load its contents into a structured pandas DataFrame with columns for "Album", "Song Name", and "Lyrics".

In [46]:
def clean_lyric_text(text):
    """A more generic function to clean common artifacts from Genius lyrics."""
    # Remove the song embed text (e.g., '123Embed')
    text = re.sub(r'\d*Embed$', '', text).strip()
    return text

# Specify the folder names created by the download script
directory_paths = ['Seedhe-Maut_Bayaan/',
                   'Seedhe-Maut_Nayaab/',
                   'Seedhe-Maut_Lunch-Break/']

# Initialize an empty DataFrame
sm_lyrics = pd.DataFrame({
    "Album": [],
    "Song Name": [],
    "Lyrics": []
})

idx = 0
for album_path in directory_paths:
    # Extract album name from the folder path
    # e.g., 'Seedhe-Maut_Bayaan/' -> 'Bayaan'
    try:
        album_name = album_path.split('_')[1][:-1].replace("-", " ")
    except IndexError:
        print(f"Warning: Could not parse album name from '{album_path}'. Skipping.")
        continue

    # Check if the directory exists
    if not Path(album_path).exists():
        print(f"Warning: Directory '{album_path}' not found. Please run the download step first.")
        continue

    for song_file in Path(album_path).glob('*.txt'):
        # Extract song name from the file path
        # e.g., '.../Pankh.txt' -> 'Pankh'
        song_name = song_file.stem.replace("-", " ")

        with open(song_file, 'r', encoding="utf-8") as f:
            # Read lines, skip the first line (which is just the song title), and join them
            lyrics_list = f.readlines()[1:]
            lyrics = ''.join(lyrics_list).replace('\n', ' ').strip()
            lyrics = clean_lyric_text(lyrics)

        # Add the data to the DataFrame
        sm_lyrics.loc[idx] = [album_name, song_name, lyrics]
        idx += 1

print(f"DataFrame created successfully with {len(sm_lyrics)} songs.")

DataFrame created successfully with 57 songs.


### Step 4: Display the Final DataFrame

Let's view the resulting DataFrame.

In [49]:
# Set display options to see the full content
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', None)

sm_lyrics

,Album,Song Name,Lyrics
0,Bayaan,Chalta Reh,"जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे, ताने दे जो भी रख भेजे में रोक नहीं सकता कोई जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) रख भेजे में रोक नहीं सकता कोई बस चाहिये नोट और है कोई चोरी नहीं, कोई चोरी नहीं नये हो पाये मोटे गड्डी छोटी नहीं एक बड़ी चंट बंदि की load है नहीं, कोई load है नहीं बस चाहिये नोट और है कोई चोर ही नहीं, कोई चोरी नही छूले तू चाँद अगर चाहे तो दूर है तू बांधे कलपाये तो क्या हुआ? क्या हुआ? एक दो दिन फुलका नहीं खाया तो दे बे तू ताली ज़माने को जेबें क्यूँ खाली कमाने दो भेजे में बात ही तू एक ही रख पैसा-पैसा-पैसा आने दो सालों पहले बाँटों ज्ञान कम कुछ पालो पहले बाँटों ज्ञान तब (Aye) सालों पहले बाँटों ज्ञान कम कुछ पालो पहले बाँटों ज्ञान तब (Aye) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे, ताने दे जो भी रख भेजे में रोक नहीं सकता कोई जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) रख भेजे में रोक नहीं सकता कोई चलता रह, चलता रह, जाने दे टोके ही नहीं सकता कोई गाने से (नहीं-नहीं) चलता रह चमकाके चमचों को रोक ही नहीं सकता कोई छाने से सड़ता तू पड़ता नहीं फरक हूँ बढ़ता मैं जानता भी (सही-सही) जलता तो ढलता रब कल का मैं रोक ही नहीं सकता कोई (नहीं-नहीं) अब-अब सारे कहे शाब्बास जो कुछ महीनों पहले रहते हमसे हताश थे दस्तूर है क्या या है प्रकृति का इलाज क्यूँकि अब फरक ना पड़े क्या सोचे हमें समाज Scene अब बढ़ेगा प्रगति की राह पे भाई अब चलेगा (अह) जबें भरेगा चाहिए ब्रहमांड सिर्फ चाँद नहीं चलेगा (यह) चाहा दिखाना, है तुझे मैंने की दुनिया में चींज जिन्हें आज़माना, चाहिए तुझे, है दुनिया खज़ाना और रहे हैं सालों से कंगाल, है ये एक जंजाल Company खरीदे सरकार भुगतती है जनता कम से कम खुद को रख थोड़ खुश जी अपने उसूलों पे हो जा मुक्त बस चलता रह, चलता रह जो साले चाहे हम बने ना कुछ, उन्हें जाने दे जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) रख भेजे में रोक नहीं सकता कोई जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, जाने दे (जा-जा, जा-जा) जाने दे, ताने दे जो भी (जा-जा, जा-जा) रख भेजे में रोक नहीं सकता कोई चलता रह, चलता रह, जाने दे टोके ही नहीं सकता कोई गाने से (नहीं-नहीं) चलता रह चमकाके चमचों को रोक ही नहीं सकता कोई छाने से सड़ता तू पड़ता नहीं फरक हूँ बढ़ता मैं जानता भी (सही-सही) जलता तो ढलता रब कल का मैं रोक ही नहीं सकता कोई (नहीं-नहीं)"
1,Bayaan,Dehshat,"Sez on the beat boy जो बोले फालतू ले लो खोप्चे में (खोप्चे में) यहाँ चले हाथी, कुत्ते भौंकते रहे चौंकते रहे सीखों सालों चौंकते रहे (चौंकते रहे) मैं खोलूँ राह, माँ बोले top पे रह झंडा गाड़ा बूंदे फाँदी दहशत है (Yo) इनको आता नहीं घंटा को झंटू वेहेम हद है पीते BP का खंभा और रहते सहमत हैं (क्यूँ) इनसे भिड़ेगा तो काका रहेम रद्द है ये बस sample है, BT है तो add जा सहम जा, बेटै सहम जा तोहफ़े में छूरी छुपाके ओढ़ेगा भेंस हाँ आ के ज़रा लेट जा मैं तेरे बिस्तर के नीचे तू बोले ""कौन है?"" मैं बोला ""Santa"" बैठी सभा में ना जाऊं खोजूँ सोना मैं, ना मैं हीरा हूँ सोचूँ कहीं चमकूँ तो तू खो जाये ना बेमानी ये ज़्यादा सही रहूँ बेनाम ही हाँ तू कहे ना हाँ में हाँ तो खायेगा का में? हाँ में हाँ होनी तो टाल होने को हुये हैं साल लेकिन अब ज़बान चखे वक्त बोले कामयाबी पास है पहले हजार चाहने वालों को भी शक था आज हटेरो को भी है मुझपे नाज़ जो बोले फालतू ले लो खोप्चे में (खोप्चे में) यहाँ चले हाथी, कुत्ते भौंकते रहे चौंकते रहे सीखों सालों चौंकते रहे (चौंकते रहे) मैं खोलूँ राह, माँ बोले top पे रह झंडा गाड़ा बूंदे फाँदी दहशत है (Yo) इनको आता नहीं घंटा को झंटू वेहेम हद है पीते BP का खंभा और रहते सहमत हैं (क्यूँ) इनसे भिड़ेगा तो काका रहेम रद्द है ये बस sample है, BT है तो add जा मुँह खो

### Step 5: Export the Lyrics as a CSV File

In [50]:
sm_lyrics.to_csv("seedhe_maut_genius_data.csv", index=False)